In [28]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import os
from os import listdir
from os.path import isfile, join

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [29]:
# parse 3D Intensity measurements
# I - Centrosomes

#100x objective
intensity_dir1 = '/Volumes/TOB_WD2/Microscopy/20230414_Centrosome_Cenp-j/3D_Intensity_gammaTub/Correct/' 
intensity_dir2 = '/Volumes/TOB_WD2/Microscopy/20230623_Centrosome_Cenp-j/3D_Intensity_gammaTub/Correct/' 
intensity_dir3 = '/Volumes/TOB_WD2/Microscopy/20230721_Centrosome_Cenp-j/3D_Intensity_gammaTub/Correct/' 
intensity_dir4 = '/Volumes/TOB_WD2/Microscopy/20230913_Centrosome_Cenp-j/3D_Intensity_gammaTub/Correct/'

def concat_centrosomes(folder, staining):
    
    average_column = 'Average_Pole_' + staining
    intdent_column = 'IntegratedDensity_Pole_' + staining
    
    dataframes = []   
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith('.'):
                filepath = root + os.sep + file 
                experiment = root.split("/")[4][:8]
                if filepath.endswith('Pole_gTub.csv'):
                    
                    single_df = pd.read_csv(filepath, encoding = 'latin-1')
                    single_df = single_df[["Average", "Cell_ID", "IntegratedDensity"]]
                    cell_id = single_df.loc[0, "Cell_ID"]
                    
                    # In case more than the centrosomes are segmented, this should only select the centrosomes
                    single_df["Single_Pole_Volume"] = single_df.IntegratedDensity / single_df.Average / 64
                    single_df = single_df.sort_values(by = "Single_Pole_Volume", ascending = False).head(2) 
                    
                    single_df[average_column] = single_df.Average.mean()
                    single_df[intdent_column] = single_df.IntegratedDensity.mean()
                    single_df["Cell_ID"] = cell_id
                    single_df['Experiment'] = experiment
                    single_df = single_df.drop(['Single_Pole_Volume', 'Average', 'IntegratedDensity'], axis = 1)
                    single_df = single_df.head(1)
                    dataframes.append(single_df) 
                else:
                    pass
            else:
                pass
    df = pd.concat(dataframes)
    return df

centrosome_df_1 = concat_centrosomes(intensity_dir1, 'gTub')
centrosome_df_2 = concat_centrosomes(intensity_dir2, 'gTub')
centrosome_df_3 = concat_centrosomes(intensity_dir3, 'gTub')
centrosome_df_4 = concat_centrosomes(intensity_dir4, 'gTub')


centrosome_df = pd.concat([centrosome_df_1, centrosome_df_2,  centrosome_df_3, centrosome_df_4])

print(centrosome_df.shape)
centrosome_df.head(1)

(570, 4)


,Cell_ID,Average_Pole_gTub,IntegratedDensity_Pole_gTub,Experiment
1,20230411_R1E309_Cenp-J_100X_gTub_Full+1uM24h_001-1,6590.641,1500878.0,20230414


In [30]:
# parse 3D Intensity measurements
# II - Total cell 

gTub_total_dir1 = '/Volumes/TOB_WD2/Microscopy/20230414_Centrosome_Cenp-j/Total_Cell_gammaTub'
gTub_total_dir2 = '/Volumes/TOB_WD2/Microscopy/20230623_Centrosome_Cenp-j/Total_Cell_gammaTub'
gTub_total_dir3 = '/Volumes/TOB_WD2/Microscopy/20230721_Centrosome_Cenp-j/Total_Cell_gammaTub'
gTub_total_dir4 = '/Volumes/TOB_WD2/Microscopy/20230913_Centrosome_Cenp-j/Total_Cell_gammaTub'


def concat_total(folder, staining):
    average_column = 'Average_Total_' + staining
    minimum_column = 'Minimum_Total_' + staining
    intdent_column = 'IntegratedDensity_Total_' + staining
    
    dataframes = []   
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = root + os.sep + file 
                if filepath.endswith('.csv'):
                    single_df = pd.read_csv(filepath, encoding = 'latin-1')
                    single_df = single_df.rename(columns = 
                                                 {
                                                     'Average': average_column, 
                                                     'Minimum': minimum_column,
                                                     'IntegratedDensity': intdent_column
                                                 }
                                                )
                    single_df = single_df[['Cell_ID', average_column, minimum_column, intdent_column]]
                    dataframes.append(single_df)
                else:
                    pass
            else:
                pass
    df = pd.concat(dataframes)
    return df

gTub_total_1 = concat_total(gTub_total_dir1, 'gTub')
gTub_total_2 = concat_total(gTub_total_dir2, 'gTub')
gTub_total_3 = concat_total(gTub_total_dir3, 'gTub')
gTub_total_4 = concat_total(gTub_total_dir4, 'gTub')
total_df = pd.concat([gTub_total_1, gTub_total_2, gTub_total_3, gTub_total_4])

print(total_df.shape)
total_df.head(1)

(588, 4)


,Cell_ID,Average_Total_gTub,Minimum_Total_gTub,IntegratedDensity_Total_gTub
0,20230411_R1E309_Cenp-J_100X_gTub_Full+1uM24h_001-1,1218.435,673,654140057


In [31]:
# parse spindle gamma tub intensities

gTub_spindle_dir1 = "/Volumes/TOB_WD2/Microscopy/20230414_Centrosome_Cenp-j/3D_Intensity_Spindle_gammaTub"
gTub_spindle_dir2 = "/Volumes/TOB_WD2/Microscopy/20230623_Centrosome_Cenp-j/3D_Intensity_Spindle_gammaTub"
gTub_spindle_dir3 = "/Volumes/TOB_WD2/Microscopy/20230721_Centrosome_Cenp-j/3D_Intensity_Spindle_gammaTub"
gTub_spindle_dir4 = "/Volumes/TOB_WD2/Microscopy/20230913_Centrosome_Cenp-j/3D_Intensity_Spindle_gammaTub"

def concat_spindles(folder, staining):
    
    average_column = 'Average_Spindle_' + staining
    minimum_column = 'Minimum_Spindle_' + staining
    intdent_column = 'IntegratedDensity_Spindle_' + staining
    
    dataframes = []   
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = root + os.sep + file 
                #print(filepath)
                if filepath.endswith('.csv'):
                    single_df = pd.read_csv(filepath, encoding = 'latin-1')
                    single_df = single_df.rename(columns = 
                                                 {
                                                     'Average': average_column, 
                                                     'Minimum': minimum_column,
                                                     'IntegratedDensity': intdent_column
                                                 }
                                                )
                    single_df = single_df[['Cell_ID', average_column, minimum_column, intdent_column]]
                    dataframes.append(single_df)
                else:
                    pass
            else:
                pass
    df = pd.concat(dataframes)
    return df

gTub_spindle_1 = concat_spindles(gTub_spindle_dir1, 'gTub')
gTub_spindle_2 = concat_spindles(gTub_spindle_dir2, 'gTub')
gTub_spindle_3 = concat_spindles(gTub_spindle_dir3, 'gTub')
gTub_spindle_4 = concat_spindles(gTub_spindle_dir4, 'gTub')


gTub_spindle_df = pd.concat([gTub_spindle_1, gTub_spindle_2, gTub_spindle_3, gTub_spindle_4])

print(gTub_spindle_df.shape)
gTub_spindle_df.head(1)

(820, 4)


,Cell_ID,Average_Spindle_gTub,Minimum_Spindle_gTub,IntegratedDensity_Spindle_gTub
0,20230411_R1E309_Cenp-J_100X_gTub_Full+1uM24h_002-1,1416.744,884,26503037


In [32]:
# parse Cell Volumes (manual segmentation)

CV_dir_1 = "/Volumes/TOB_WD2/Microscopy/20230414_Centrosome_Cenp-j/Cell_Volumes/"
CV_dir_2 = "/Volumes/TOB_WD2/Microscopy/20230623_Centrosome_Cenp-j/Cell_Volumes/"
CV_dir_3 = "/Volumes/TOB_WD2/Microscopy/20230721_Centrosome_Cenp-j/Cell_Volumes/"
CV_dir_4 = "/Volumes/TOB_WD2/Microscopy/20230913_Centrosome_Cenp-j/Cell_Volumes/"

def concat_CV(folder):
    CV_dataframes = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = root + os.sep + file  
                if filepath.endswith(".csv"):
                    single_df = pd.read_csv(filepath)
                    single_df = single_df.rename(
                            columns = {
                                "Volume": "Cell_Volume_um3",
                                "SurfaceArea": "Cell_Surface_Area_um2"
                            }
                    )
                    single_df = single_df[["Cell_ID", "Cell_Volume_um3", "Cell_Surface_Area_um2"]]

                    CV_dataframes.append(single_df)
                else:
                    pass
            else:
                pass
    CV_df = pd.concat(CV_dataframes)
    return CV_df 

CV_df_1 = concat_CV(CV_dir_1)
CV_df_2 = concat_CV(CV_dir_2)
CV_df_3 = concat_CV(CV_dir_3)
CV_df_4 = concat_CV(CV_dir_4)

CV_df = pd.concat([CV_df_1, CV_df_2, CV_df_3, CV_df_4])

print(CV_df.shape)
CV_df.head(1)

(588, 3)


,Cell_ID,Cell_Volume_um3,Cell_Surface_Area_um2
0,20230411_R1E309_Cenp-J_100X_gTub_Full+1uM24h_001-1,3057.223,1145.108


In [33]:
# parse Centrosome Volumes 
# Note: This is the combined volume of both centrosomes

CentV_dir_1 = "/Volumes/TOB_WD2/Microscopy/20230414_Centrosome_Cenp-j/GammaTub_Volumes"
CentV_dir_2 = "/Volumes/TOB_WD2/Microscopy/20230623_Centrosome_Cenp-j/GammaTub_Volumes"
CentV_dir_3 = "/Volumes/TOB_WD2/Microscopy/20230721_Centrosome_Cenp-j/GammaTub_Volumes"
CentV_dir_4 = "/Volumes/TOB_WD2/Microscopy/20230913_Centrosome_Cenp-j/GammaTub_Volumes"

def concat_CentV(folder, target):
    CentV_dataframes = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if not file.startswith("."):
                filepath = root + os.sep + file  
                if filepath.endswith(".csv"):
                    single_df = pd.read_csv(filepath)
                    single_df = single_df.rename(
                            columns = {
                                "Volume": "Centrosome_Volume_um3",
                            }
                    )
                    single_df = single_df[["Cell_ID", "Centrosome_Volume_um3"]]
                    single_df["Target"] = target
                    CentV_dataframes.append(single_df)
                else:
                    pass
            else:
                pass
    CentV_df = pd.concat(CentV_dataframes)
    return CentV_df 

CentV_df_1 = concat_CentV(CentV_dir_1, target = "gTub")
CentV_df_2 = concat_CentV(CentV_dir_2, target = "gTub")
CentV_df_3 = concat_CentV(CentV_dir_3, target = "gTub")
CentV_df_4 = concat_CentV(CentV_dir_4, target = "gTub")

CentV_df = pd.concat([CentV_df_1, CentV_df_2, CentV_df_3, CentV_df_4])

print(CentV_df.shape)
CentV_df.head(1)

(588, 3)


,Cell_ID,Centrosome_Volume_um3,Target
0,20230411_R1E309_Cenp-J_100X_gTub_Full+1uM24h_001-1,2.705,gTub


In [34]:
# parse Spindle3D morphometry
# parse Plugin Measurements

input_dir1 = "/Volumes/TOB_WD2/Microscopy/20230414_Centrosome_Cenp-j/Spindle3D/309/Correct"
input_dir2 = "/Volumes/TOB_WD2/Microscopy/20230623_Centrosome_Cenp-j/Spindle3D/Correct"
input_dir3 = "/Volumes/TOB_WD2/Microscopy/20230721_Centrosome_Cenp-j/Spindle3D/Correct"
input_dir4 = "/Volumes/TOB_WD2/Microscopy/20230913_Centrosome_Cenp-j/Spindle3D/Correct"

spindle_input_folders = [
    input_dir1, input_dir2, input_dir3, input_dir4
]

def concat_spindle3d(folderlist):
    dataframes = []
    for folder in folderlist:    
        for root, dirs, files in os.walk(folder):
            for file in files:
                if not file.startswith("."):
                    filepath = root + os.sep + file  
                    if filepath.endswith(".txt"):
                        single_df = pd.read_csv(filepath, sep = " ", delimiter = "\t", encoding = 'unicode_escape')
                        dataframes.append(single_df)
                    else:
                        pass
                else:
                    pass
    df = pd.concat(dataframes)
    return df

spindle_df = concat_spindle3d(spindle_input_folders)

print("Finished parsing data.")

def get_nakedname(path):
    # parse cell ids from file name in spindle3D table
    base = os.path.basename(str(path))
    naked = os.path.splitext(base)[0]
    return naked

# modify spindle 3d dataframe
spindle_df["Cell_ID"] = spindle_df.Path_InputImage.apply(get_nakedname)

subselection_spindle = [
    "Cell_ID",
    "Spindle_Angle_Degrees",  
    "Spindle_Length_um", 
    "Spindle_Volume_um3", 
    "Spindle_Width_Avg_um", 
    "Tubulin_Spindle_Average_Intensity",
    "Chromatin_Volume_um3",
    "MetaphasePlate_Length_um",
    "MetaphasePlate_Width_um",
    "Version"
]
spindle_df = spindle_df[subselection_spindle]
print(spindle_df.shape)
spindle_df.head(1)

Finished parsing data.
(515, 10)


,Cell_ID,Spindle_Angle_Degrees,Spindle_Length_um,Spindle_Volume_um3,Spindle_Width_Avg_um,Tubulin_Spindle_Average_Intensity,Chromatin_Volume_um3,MetaphasePlate_Length_um,MetaphasePlate_Width_um,Version
0,20230411_R1E309_Cenp-J_100X_gTub_Full+1uM24h_002-1,4.290449,11.073617,292.296875,8.065789,1630.458866,418.0,14.0,2.25,Spindle3D version: 0.8.0


In [35]:
# merge dataframes
spindle_df = spindle_df.merge(gTub_spindle_df, how = "outer")
df = centrosome_df.merge(total_df).merge(CV_df).merge(CentV_df).merge(spindle_df, how = "outer")

print(df.shape)

(820, 13)
(890, 23)


In [36]:
# Finalise Dataframe

def correct_condition(x):
    if x == "1uM48h":
        new_x = "Full+1uM48h"
        return new_x
    else:
        return x

df["Cell_line"] = df.Cell_ID.str.split("_").str.get(1)
df["Condition"] = df.Cell_ID.str.split("_").str.get(5)
df["Condition"] = df.Condition.apply(correct_condition)

In [38]:
def rename_condition(x):
    if x == "Full+DMSO":
        return "1_DMSO"
    elif x == "Full+1uM24h":
        return "2_1µM"
    elif x == "Full+1uM48h":
        return "2_1µM"
    elif x == "CCB5uM":
        return "4_5µM"
    elif x == "Full+CCB5uM":
        return "4_5µM"
    elif x == "CCB10uM":
        return "5_10µM"
    elif x == "Full+CCB10uM":
        return "5_10µM"
    elif x == "Full+CCB20uM":
        return "6_20µM"
    elif x == "N2B27+RA":
        return "7_N2B27+RA"
    else:
        pass

df["Condition"] = df.Condition.apply(rename_condition)


# Offset subtraction based on minimum inside the cell binary mask
df["Average_Pole_gTub"] = df.Average_Pole_gTub - df.Minimum_Total_gTub
df["Average_Total_gTub"] = df.Average_Total_gTub - df.Minimum_Total_gTub
df["Average_Spindle_gTub"] = df.Average_Spindle_gTub - df.Minimum_Total_gTub

# Fluorescent masses
df["Average_gTub_Poles_norm"] = df["Average_Pole_gTub"] / df["Average_Total_gTub"] # gTub concentration (average signal) in poles normalised by total gTub concentration (total average signal)

df["Average_gTub_Spindle_norm"] = df["Average_Spindle_gTub"] / df["Average_Total_gTub"] # gTub concentration (average signal) within spindle volume normalised by total gTub concentration (total average signal)

df["gTub_mass_poles_norm"] = df["Average_gTub_Poles_norm"] * df["Centrosome_Volume_um3"]
df["gTub_mass_at_poles_percent"] = (df["Average_Pole_gTub"] * df["Centrosome_Volume_um3"]) / (df["Average_Total_gTub"] * df["Cell_Volume_um3"]) * 100

df["gTub_mass_Spindle_norm"] = df["Average_gTub_Spindle_norm"] * df["Spindle_Volume_um3"]
df["gTub_mass_SpindleOnly"] = df["gTub_mass_Spindle_norm"] - df["gTub_mass_poles_norm"]

df["gTub_Centrosome_vs_Spindle"] = df["gTub_mass_poles_norm"] / df["gTub_mass_SpindleOnly"]

df["Centrosome_Occupancy"] = df.Centrosome_Volume_um3 / df.Cell_Volume_um3 * 100
df["Centrosome_to_spindle_ratio"] = df.Centrosome_Volume_um3 / df.Spindle_Volume_um3 
df["Spindle_Occupancy"] = df.Spindle_Volume_um3 / df.Cell_Volume_um3 * 100

In [40]:
# Export Dataframe

destination = "/Volumes/TOB_WD2/Image_Analysis/CCB02/Dataframes" + "/MasterDataFrame_CCB02_gTub.csv"
df.to_csv(destination)
print("Successfully saved dataframe at {}".format(destination))

Successfully saved dataframe at /Volumes/TOB_WD2/Image_Analysis/CCB02/Dataframes/MasterDataFrame_CCB02_gTub.csv
